<a href="https://colab.research.google.com/github/Renan2004a/Avalia-o-de-LLMs-na-constru-o-de-Casos-de-Teste-de-Software/blob/main/Metodologia_de_Avalia%C3%A7%C3%A3o_dos_Testes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter

# ========== CONFIGURAÇÃO ==========
arquivos_llms = {
    'Claude_Sonnet': 'Claude_Sonnet_4_5.py',
    'DeepSeek_V3': 'DeepSeek-V3.py',
    'Copilot_Smart': 'Copilot_Smart _(GPT‑5).py',
    'Gemini_1_5': 'Gemini_1_5_Flash.py',
    'GPT_4o_mini': 'GPT-4o_mini.py'
}

# ========== FUNÇÕES DE ANÁLISE ==========

def analisar_arquivo_testes(caminho_arquivo):
    """Analisa um arquivo de testes e extrai métricas"""
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            conteudo = f.read()

        metrics = {
            'total_testes': 0,
            'testes_sucesso': 0,
            'testes_erro': 0,
            'testes_borda': 0,
            'testes_metodo': 0,
            'cobertura_cenarios': set(),
            'complexidade_testes': 0,
            'comentarios_por_teste': 0,
            'linhas_por_teste': 0,
            'asserts_por_teste': 0,
            'erros_cobertos': set(),
            'metodos_cobertos': set()
        }

        # Análise por regex para identificar padrões de teste
        padrao_funcao_teste = r'def (test_[^(]+)\([^)]*\):'
        funcoes_teste = re.findall(padrao_funcao_teste, conteudo)
        metrics['total_testes'] = len(funcoes_teste)

        # Análise de cenários cobertos
        for func in funcoes_teste:
            func_lower = func.lower()

            # Classificar tipo de teste
            if any(palavra in func_lower for palavra in ['sucesso', 'valido', 'happy']):
                metrics['testes_sucesso'] += 1
            elif any(palavra in func_lower for palavra in ['erro', 'invalido', 'error', 'validation']):
                metrics['testes_erro'] += 1
                # Extrair tipo de erro
                if 'nome' in func_lower:
                    metrics['erros_cobertos'].add('nome')
                elif 'email' in func_lower:
                    metrics['erros_cobertos'].add('email')
                elif 'idade' in func_lower:
                    metrics['erros_cobertos'].add('idade')
                elif 'ativo' in func_lower:
                    metrics['erros_cobertos'].add('ativo')
                elif 'duplicado' in func_lower:
                    metrics['erros_cobertos'].add('id_duplicado')
            elif any(palavra in func_lower for palavra in ['borda', 'limite', 'edge']):
                metrics['testes_borda'] += 1
            elif any(palavra in func_lower for palavra in ['metodo', 'criar', 'buscar', 'atualizar', 'excluir']):
                metrics['testes_metodo'] += 1
                # Identificar método coberto
                if 'criar' in func_lower:
                    metrics['metodos_cobertos'].add('criarUsuario')
                elif 'buscar' in func_lower:
                    metrics['metodos_cobertos'].add('buscarUsuario')
                elif 'atualizar' in func_lower:
                    metrics['metodos_cobertos'].add('atualizarUsuario')
                elif 'excluir' in func_lower:
                    metrics['metodos_cobertos'].add('excluirUsuario')

            # Cenários cobertos
            if 'nome' in func_lower:
                metrics['cobertura_cenarios'].add('validação_nome')
            if 'email' in func_lower:
                metrics['cobertura_cenarios'].add('validação_email')
            if 'idade' in func_lower:
                metrics['cobertura_cenarios'].add('validação_idade')
            if 'ativo' in func_lower:
                metrics['cobertura_cenarios'].add('validação_ativo')
            if 'id' in func_lower and 'duplicado' in func_lower:
                metrics['cobertura_cenarios'].add('id_duplicado')

        # Análise de complexidade (contagem de asserts)
        asserts = re.findall(r'assert ', conteudo)
        metrics['asserts_por_teste'] = len(asserts) / metrics['total_testes'] if metrics['total_testes'] > 0 else 0

        # Análise de comentários
        comentarios = re.findall(r'""".*?"""|\'\'\'.*?\'\'\'|#.*?$', conteudo, re.MULTILINE | re.DOTALL)
        metrics['comentarios_por_teste'] = len(comentarios) / metrics['total_testes'] if metrics['total_testes'] > 0 else 0

        # Converter sets para contagens (mantemos os sets originais também para análise qualitativa)
        metrics['num_erros_cobertos'] = len(metrics['erros_cobertos'])
        metrics['num_metodos_cobertos'] = len(metrics['metodos_cobertos'])
        metrics['num_cobertura_cenarios'] = len(metrics['cobertura_cenarios'])

        return metrics

    except Exception as e:
        print(f"Erro ao analisar {caminho_arquivo}: {e}")
        return None

# ========== ANÁLISE COMPARATIVA ==========

def calcular_pontuacao_final(metrics_dict, resultados_completos):
    """Calcula uma pontuação final baseada em múltiplas métricas"""
    if not metrics_dict:
        return 0

    pesos = {
        'total_testes': 0.15,
        'num_cobertura_cenarios': 0.20,
        'testes_erro': 0.15,
        'testes_borda': 0.15,
        'num_metodos_cobertos': 0.10,
        'num_erros_cobertos': 0.10,
        'asserts_por_teste': 0.10,
        'comentarios_por_teste': 0.05
    }

    # Encontrar valores máximos para normalização
    valores_validos = [m for m in resultados_completos.values() if m is not None]

    max_testes = max(m['total_testes'] for m in valores_validos) if valores_validos else 1
    max_cenarios = max(m['num_cobertura_cenarios'] for m in valores_validos) if valores_validos else 1
    max_erros = max(m['testes_erro'] for m in valores_validos) if valores_validos else 1
    max_borda = max(m['testes_borda'] for m in valores_validos) if valores_validos else 1
    max_metodos = max(m['num_metodos_cobertos'] for m in valores_validos) if valores_validos else 1
    max_erros_cobertos = max(m['num_erros_cobertos'] for m in valores_validos) if valores_validos else 1
    max_asserts = max(m['asserts_por_teste'] for m in valores_validos) if valores_validos else 1
    max_comentarios = max(m['comentarios_por_teste'] for m in valores_validos) if valores_validos else 1

    pontuacao = 0
    if max_testes > 0:
        pontuacao += pesos['total_testes'] * (metrics_dict['total_testes'] / max_testes)
    if max_cenarios > 0:
        pontuacao += pesos['num_cobertura_cenarios'] * (metrics_dict['num_cobertura_cenarios'] / max_cenarios)
    if max_erros > 0:
        pontuacao += pesos['testes_erro'] * (metrics_dict['testes_erro'] / max_erros)
    if max_borda > 0:
        pontuacao += pesos['testes_borda'] * (metrics_dict['testes_borda'] / max_borda)
    if max_metodos > 0:
        pontuacao += pesos['num_metodos_cobertos'] * (metrics_dict['num_metodos_cobertos'] / max_metodos)
    if max_erros_cobertos > 0:
        pontuacao += pesos['num_erros_cobertos'] * (metrics_dict['num_erros_cobertos'] / max_erros_cobertos)
    if max_asserts > 0:
        pontuacao += pesos['asserts_por_teste'] * (metrics_dict['asserts_por_teste'] / max_asserts)
    if max_comentarios > 0:
        pontuacao += pesos['comentarios_por_teste'] * (metrics_dict['comentarios_por_teste'] / max_comentarios)

    return pontuacao * 100  # Converter para porcentagem

# ========== EXECUTAR ANÁLISE ==========

print("Iniciando análise dos arquivos de teste...")
resultados = {}
for llm, arquivo in arquivos_llms.items():
    print(f"Analisando {llm}...")
    resultados[llm] = analisar_arquivo_testes(arquivo)

# Criar DataFrame com resultados (usando as métricas numéricas)
dados_para_dataframe = {}
for llm, metrics in resultados.items():
    if metrics:
        dados_para_dataframe[llm] = {
            'total_testes': metrics['total_testes'],
            'testes_sucesso': metrics['testes_sucesso'],
            'testes_erro': metrics['testes_erro'],
            'testes_borda': metrics['testes_borda'],
            'testes_metodo': metrics['testes_metodo'],
            'cobertura_cenarios': metrics['num_cobertura_cenarios'],
            'asserts_por_teste': metrics['asserts_por_teste'],
            'comentarios_por_teste': metrics['comentarios_por_teste'],
            'metodos_cobertos': metrics['num_metodos_cobertos'],
            'erros_cobertos': metrics['num_erros_cobertos']
        }

df_metrics = pd.DataFrame.from_dict(dados_para_dataframe, orient='index')
df_metrics = df_metrics.fillna(0)

# Calcular pontuações finais
pontuacoes = {}
for llm in df_metrics.index:
    metrics_dict = resultados[llm]
    if metrics_dict:
        pontuacoes[llm] = calcular_pontuacao_final(metrics_dict, resultados)
    else:
        pontuacoes[llm] = 0

df_metrics['pontuacao_final'] = df_metrics.index.map(pontuacoes)

# Ordenar por pontuação
df_metrics = df_metrics.sort_values('pontuacao_final', ascending=False)

print("\n" + "="*80)
print("RESULTADO DA COMPARAÇÃO DE LLMs")
print("="*80)
print(df_metrics.round(2))

# ========== VISUALIZAÇÃO DETALHADA ==========

print("\n" + "="*80)
print("ANÁLISE DETALHADA POR LLM")
print("="*80)

for llm in df_metrics.index:
    metrics = resultados[llm]
    if metrics:
        print(f"\n--- {llm} ---")
        print(f"Pontuação Final: {df_metrics.loc[llm, 'pontuacao_final']:.1f}%")
        print(f"Total de Testes: {metrics['total_testes']}")
        print(f"Testes de Sucesso: {metrics['testes_sucesso']}")
        print(f"Testes de Erro: {metrics['testes_erro']}")
        print(f"Testes de Borda: {metrics['testes_borda']}")
        print(f"Testes por Método: {metrics['testes_metodo']}")
        print(f"Cenários Cobertos: {metrics['num_cobertura_cenarios']}")
        print(f"Métodos Cobertos: {metrics['num_metodos_cobertos']}/4")
        print(f"Tipos de Erro Cobertos: {metrics['num_erros_cobertos']}/5")
        print(f"Asserts por Teste: {metrics['asserts_por_teste']:.1f}")
        print(f"Comentários por Teste: {metrics['comentarios_por_teste']:.1f}")

# ========== RANKING FINAL ==========

print("\n" + "="*80)
print("🏆 RANKING FINAL DOS LLMs")
print("="*80)

ranking = df_metrics['pontuacao_final'].sort_values(ascending=False)
for i, (llm, score) in enumerate(ranking.items(), 1):
    print(f"{i}º Lugar: {llm} - {score:.1f}%")

# ========== ANÁLISE QUALITATIVA ==========

print("\n" + "="*80)
print("📊 ANÁLISE QUALITATIVA")
print("="*80)

# Verificar cobertura completa dos métodos
metodos_esperados = {'criarUsuario', 'buscarUsuario', 'atualizarUsuario', 'excluirUsuario'}
for llm in df_metrics.index:
    metrics = resultados[llm]
    if metrics:
        metodos_cobertos = metrics['metodos_cobertos']  # Este é o set original
        cobertura = len(metodos_cobertos) / len(metodos_esperados) * 100
        print(f"{llm}: {cobertura:.0f}% de cobertura de métodos ({len(metodos_cobertos)}/{len(metodos_esperados)})")
        if metodos_cobertos:
            print(f"  Métodos testados: {', '.join(sorted(metodos_cobertos))}")

# Verificar diversidade de testes
print(f"\n📈 Diversidade de Testes (Erro/Sucesso/Borda):")
for llm in df_metrics.index:
    metrics = resultados[llm]
    if metrics:
        total = metrics['total_testes']
        if total > 0:
            pct_erro = (metrics['testes_erro'] / total) * 100
            pct_sucesso = (metrics['testes_sucesso'] / total) * 100
            pct_borda = (metrics['testes_borda'] / total) * 100
            outros = 100 - pct_erro - pct_sucesso - pct_borda
            print(f"{llm}: Erro({pct_erro:.0f}%) Sucesso({pct_sucesso:.0f}%) Borda({pct_borda:.0f}%) Outros({outros:.0f}%)")

# Análise de cenários cobertos
print(f"\n🎯 Cenários de Validação Cobertos:")
cenarios_esperados = {'validação_nome', 'validação_email', 'validação_idade', 'validação_ativo', 'id_duplicado'}
for llm in df_metrics.index:
    metrics = resultados[llm]
    if metrics:
        cenarios_cobertos = metrics['cobertura_cenarios']
        cobertura = len(cenarios_cobertos) / len(cenarios_esperados) * 100
        print(f"{llm}: {cobertura:.0f}% de cenários ({len(cenarios_cobertos)}/{len(cenarios_esperados)})")
        if cenarios_cobertos:
            print(f"  Cenários: {', '.join(sorted(cenarios_cobertos))}")

# Resumo final
print("\n" + "="*80)
print("🎯 RESUMO DAS FORÇAS E FRAQUEZAS")
print("="*80)

for llm in df_metrics.index:
    metrics = resultados[llm]
    if metrics:
        print(f"\n{llm}:")
        pontos_fortes = []
        pontos_fracos = []

        if metrics['total_testes'] >= 20:
            pontos_fortes.append("Alta quantidade de testes")
        elif metrics['total_testes'] < 10:
            pontos_fracos.append("Poucos testes")

        if metrics['num_metodos_cobertos'] == 4:
            pontos_fortes.append("Cobertura completa de métodos")
        else:
            pontos_fracos.append(f"Cobertura parcial de métodos ({metrics['num_metodos_cobertos']}/4)")

        if metrics['num_erros_cobertos'] >= 4:
            pontos_fortes.append("Boa cobertura de erros")
        elif metrics['num_erros_cobertos'] < 3:
            pontos_fracos.append("Cobertura limitada de erros")

        if metrics['asserts_por_teste'] >= 2:
            pontos_fortes.append("Testes robustos (muitos asserts)")
        elif metrics['asserts_por_teste'] < 1:
            pontos_fracos.append("Testes pouco assertivos")

        if pontos_fortes:
            print(f"  ✅ Forças: {', '.join(pontos_fortes)}")
        if pontos_fracos:
            print(f"  ❌ Fraquezas: {', '.join(pontos_fracos)}")